In [2]:
!pip install sklearn_pandas
!pip install git+https://github.com/jpmml/sklearn2pmml.git

  Cloning https://github.com/jpmml/sklearn2pmml.git to /tmp/pip-1pxq8m85-build
  Running setup.py install for sklearn2pmml ... - \ done


# Scikit-learn demo

## Build pipeline

In [3]:
import pandas as pd
import numpy as np
import urllib.request
import urllib.parse
import json

from sklearn.datasets import load_diabetes,load_iris
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse, r2_score
from sklearn2pmml import PMMLPipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn2pmml import sklearn2pmml

diabetes = load_diabetes()
model = LinearRegression()
model.fit(diabetes.data, diabetes.target)
expected  = diabetes.target
predicted = model.predict(diabetes.data)
print("Mean Squared Error: %0.3f" % mse(expected, predicted))
print("Coefficient of Determination: %0.3f" % r2_score(expected, predicted))

iris = load_iris()
iris_df = pd.DataFrame(iris.data,columns=iris.feature_names)
iris_df['Species'] = iris.target
iris_pipeline = PMMLPipeline([
    ("classifier", DecisionTreeClassifier())
])
iris_pipeline.fit(iris_df[iris_df.columns.difference(["Species"])], iris_df["Species"])
sklearn2pmml(iris_pipeline, "DecisionTreeIris.pmml", with_repr = True)
pmmlBytes = bytearray(open('DecisionTreeIris.pmml', 'rb').read())

Mean Squared Error: 2859.690
Coefficient of Determination: 0.518


### Serve

In [9]:
update_url = "http://<your-ip>:39040/update-pmml/pmml_iris" % url
update_headers = {}
update_headers["Content-type"] = "application/xml"
req = urllib.request.Request(update_url,headers=update_headers,data=pmmlBytes)
resp = urllib.request.urlopen(req)
print(resp.status)

TypeError: not all arguments converted during string formatting

### Evaluate

In [8]:
evaluate_url = 'http://<your-ip>:39040/evaluate-pmml/pmml_iris' % url
evaluate_headers = {}
evaluate_headers['Content-type'] = 'application/json'
test = iris_df
input_params = test.ix[0,:-1].to_json()
encoded_input_params = input_params.encode('utf-8')
req = urllib.request.Request(evaluate_url, headers=evaluate_headers,data=encoded_input_params)
resp = urllib.request.urlopen(req)
print(resp.read())

TypeError: not all arguments converted during string formatting